## BERT - Encoder stack of transformer architecture. 
[개념](https://www.kaggle.com/ritesh2000/bert-all-in-one)  
[코드](https://www.kaggle.com/dionysios1981/bert-pytorch-tweets)

bert를 사용하는 이유

1. 데이터의 부족. -> 이번 데이터는 너무 많아서 줄였다. 
2. 적은 데이터로도 큰 결과.  
3. 전이학습. -> pre-trained 돼서 훨씬 사용이 간편하다고 한다. 


bert의 t는 transformer이고 이 변환은 기존의 rnn의 방식인 한칸이 아니라 서로 상호작용을 할 수 있게 했다. 

방법 

전처리 - tokenizer 
input으로 바뀐후 세개로 나눠 들어간다.

token embedding. segment embedding, positional embedding. 

특징은 mask 인데 

mask가 뭐냐. 쉽게 말하면 가리고 자리 맞추기 훈련해보는.  

나중에 padding 해서 글자길이 맞춰주는데 padding으로 인한 token은 0이다. 의미없어으니까 mask를 씌워서 mask가 된곳만 훈련을 하는 그런식으로. 그리고 코드를 보며 설명.

In [41]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [43]:
train = pd.read_csv('/kaggle/input/sejong-ai-challenge-p2/train.csv',usecols=[1,2])
test= pd.read_csv('/kaggle/input/sejong-ai-challenge-p2/test.csv',usecols=[1])
sample_submission=pd.read_csv('/kaggle/input/sejong-ai-challenge-p2/sample_submission.csv')

In [44]:
# 시간 너무 오래걸려서 
train=train.sample(frac=0.1)

In [46]:
train.isna().sum()

# nan 값 확인.
# train=train.dropna()

Text     0
Label    0
dtype: int64

In [47]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [48]:
class TrainDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        # tokenizer는 str로 받는다. 혹은 list of str. 
        text = self.texts[item]
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            text,
            # cls와 sep등 token을 넣어준다. bert 규칙.
            add_special_tokens=True,
            # 그런데 max_length를 정해줬으니까 절단을 시켜줘야한다. 
            max_length=self.max_len,
            return_token_type_ids=False,
            # truncation 이 절단이데 절단 안시키면 512로 그대로 들어간다.
            truncation=True,
            # 그리고 이건 바뀐부분 같은데 pad_to_max_length가 예전에 truncation과 max_length를 같이 했다면
            # padding 자체는 이런기능이 없어서 나눠줘야 하는것 같다. 
            padding='max_length',
            # return attetion_mask는 주목할 mask 를 알려주는것.
            return_attention_mask=True,
            # tensor를 알려준다. pt 는 pytorch tensor. tf는 tensorflow numpy는 np
            return_tensors='pt'
        )
        
        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

In [49]:
class BERT(nn.Module):
    
    def __init__(self):
        super(BERT, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')       
        self.fc = nn.Linear(768,2)
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = output["pooler_output"]
        return self.fc(output)

In [50]:
df_train,df_test=train_test_split(train,test_size=0.2,random_state=71)

In [51]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = TrainDataset(texts=df.Text.to_numpy(),targets=df.Label.to_numpy(),
                      tokenizer=tokenizer,max_len=max_len)    
    return DataLoader(ds,batch_size=batch_size)

MAX_LEN=64
# 빠르게 간만 보려고 512했는데 oom, 256도 oom 
BATCH_SIZE = 128
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [52]:
model=BERT().to(device)
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss().to(device)

In [53]:
best_loss = np.inf
for epoch in range(3):
    model.train()
    for d in train_data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        
        loss.backward()
        # 그래디언트 폭주 막기 위해 쓴다고 한다.
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    model = model.eval()
    valid_loss = 0
    with torch.no_grad():
        for d in val_data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
             )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            
            valid_loss += loss.item()
            valid_loss /= len(val_data_loader)
    print(f"EPOCH:{epoch}, Loss:{valid_loss}")
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), "nlpmodel.pth")
        print("saved...")

EPOCH:0, Loss:0.003396518323073142
saved...
EPOCH:1, Loss:0.002583795433797997
saved...
EPOCH:2, Loss:0.0019358544131108048
saved...


In [54]:
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
         )
        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten()
          }

In [55]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = TestDataset(texts=test.Text.to_numpy(),tokenizer=tokenizer,max_len=max_len)
    return DataLoader(ds,batch_size=batch_size)

MAX_LEN=64
BATCH_SIZE = 64
test_data_loader = create_data_loader(test, tokenizer, MAX_LEN, BATCH_SIZE)

In [56]:
model = model.to(device)
model.load_state_dict(torch.load("./nlpmodel.pth"))

<All keys matched successfully>

In [57]:
submit_preds = []

model.eval()
with torch.no_grad():
    for d in test_data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
            
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
             )
        _, preds = torch.max(outputs, dim=1)
        submit_preds.extend(preds)
    predictions = torch.stack(submit_preds).cpu()


In [58]:
pred=predictions.cpu().detach().numpy()
sample_submission["Label"] = pred
print(sample_submission.head())

   id  Label
0   0      1
1   1      0
2   2      0
3   3      0
4   4      0


In [ ]:
# sample_submission.to_csv("submit_bert.csv", index=False)